In [5]:
setupServices()

A Koin Application has already been started
org.koin.core.error.KoinAppAlreadyStartedException: A Koin Application has already been started
	at org.koin.core.context.GlobalContext.register(GlobalContext.kt:44)
	at org.koin.core.context.GlobalContext.startKoin(GlobalContext.kt:63)
	at org.koin.core.context.DefaultContextExtKt.startKoin(DefaultContextExt.kt:40)
	at MainKt.setupServices(Main.kt:47)
	at Line_8_jupyter.<init>(Line_8.jupyter.kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvalu

In [ ]:
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

In [7]:
import core.githubAdvisory.GhsaDataProvider
import core.projectKb.KbDataProvider
import org.koin.core.component.KoinComponent
import org.koin.core.component.get

class K: KoinComponent {
    val kbDataProvider = get<KbDataProvider>()
    val ghsaDataProvider = get<GhsaDataProvider>()
}

val k = K()

In [8]:
val statements = k.kbDataProvider.getStatements()
statements.count()

1297

In [9]:
DISPLAY(statements.filter { it.artifacts != null }.count())
DISPLAY(statements.filter { it.fixes != null }.count())
DISPLAY(statements.filter { it.artifacts != null && it.fixes != null }.count())

617

1215

535

In [10]:
val ghsaJsonData = k.ghsaDataProvider.getJsonData()
ghsaJsonData.count()

3989

In [11]:
ghsaJsonData.filter { it.affected.count() > 0 }.count()

3989

In [12]:
data class ResultDataItem(
    val cveId: String?,
    val sapKbVulnId: String?,
    val sapKbUrl: String?,
    val ghsaUrl: String?,
    val ghsaId: String?,
    val commitUrl: String?,
    val latestVulnGav: String?,
    val earliestFixedGav: String?,
    val affectedSourcePath: String?,
    val latestVulnJarPath: String?,
    val earliestFixedJarPath: String?,
    val affectedClassFilePath: String?,
    val affectedClassFileName: String?,
)

In [13]:
val kbCveToStatement = statements.groupBy { it.vulnerabilityID }.map { it.key to it.value.first() }.toMap()

In [14]:
val kbGhsaData = ghsaJsonData.map {
        val matchedCve = it.aliases.filter { kbCveToStatement.keys.contains(it) }.firstOrNull()
        if (matchedCve == null)
            return@map null
        Pair(matchedCve, it)
    }.filterNotNull()
    .map {
        Pair(it.first, Pair(it.second, kbCveToStatement[it.first]!!))        
    }.toMap()
    
kbGhsaData.count()

666

In [15]:
// ranges array in GHSA always has only 1 element (or its null)
kbGhsaData.forEach {
    it.value.first.affected.forEach {
        if (it.ranges != null && it.ranges!!.count() != 1) {
            println("should not be possible")
        }
    }
}

In [16]:
kbGhsaData.filter { 
    it.value.first.affected.any { it.ranges == null && it.versions != null }
 }.map {
     println(it.key)
     it
 }
 .count()

CVE-2019-12402
CVE-2018-10237
CVE-2021-44228
CVE-2018-1131


4

CVE-2021-44228 (log4j) records in both GHSA and SAP-KB mentions artifacts from projects other than the original apache repository. But the commits are only from apache repository. 

In [17]:
// how many records will be left if we only consider single-commit-single-affected-package?
DISPLAY("total: " + kbGhsaData.count())

DISPLAY(
    "single affected: " + kbGhsaData.filter {
        it.value.first.affected.count() == 1
    }.count()
)

DISPLAY(
    "single branch: " + kbGhsaData.filter {
        it.value.second.fixes != null && it.value.second.fixes!!.count() == 1
    }.count()
)

DISPLAY("single commit: " + kbGhsaData.filter {
    it.value.second.fixes != null && it.value.second.fixes!!.count() == 1
}.filter {
    it.value.second.fixes!!.first().commits.count() == 1
}.count())

DISPLAY("single affected & single branch: " + kbGhsaData.filter {
    it.value.second.fixes != null && it.value.second.fixes!!.count() == 1
}.filter {
    it.value.first.affected.count() == 1
}.count()
)

DISPLAY("single affected & single commit: " + kbGhsaData.filter {
    it.value.second.fixes != null && it.value.second.fixes!!.count() == 1
}.filter {
    it.value.second.fixes!!.first().commits.count() == 1
}.filter {
    it.value.first.affected.count() == 1
}
.count())

total: 666

single affected: 365

single branch: 481

single commit: 385

single affected & single branch: 305

single affected & single commit: 270

We will continue with "single affected & single commit" records

First let's investigate what actually "last_affected" means. 

In [18]:
val singularKbGhsaData = kbGhsaData.filter {
    it.value.second.fixes != null && it.value.second.fixes!!.count() == 1
}.filter {
    it.value.second.fixes!!.first().commits.count() == 1
}.filter {
    it.value.first.affected.count() == 1
}
//.filter {
//    val e = it.value.first.affected.first().ranges!!.first().events
//    val intro = e.first { it.introduced != null }.introduced
//    val lastAffected = e.firstOrNull() { it.lastAffected != null }?.lastAffected
//    val fixed = e.firstOrNull() { it.fixed != null }?.fixed
//    intro == "0"
//}
.filter {
    it.value.first.affected.first().ranges!!.first().events.any { e -> e.lastAffected != null }
}

DISPLAY(singularKbGhsaData.count())
//DISPLAY(singularKbGhsaData.toList().random())

15

It does not have a clear and coherent meaning. It seems most likely to be the last version known to be vulnerable at the time of adding the record in GHSA. 

Notes:
- During the investigation I have seen a few records with wrong package name. For example: org.apache.struts:struts-master
- There are so many records with the "introduced" field value of "0". I have cross-checked them with snyk database and they seem to be wrong. There are actual introduced versions in snyk database that is not recorded in GHSA.

OK. Let's continue with the "single affected & single commit" records without the ones that have "last_affected"

In [19]:
val singularKbGhsaData = kbGhsaData.filter {
    it.value.second.fixes != null && it.value.second.fixes!!.count() == 1
}.filter {
    it.value.second.fixes!!.first().commits.count() == 1
}.filter {
    it.value.first.affected.count() == 1
}
.filter {
    it.value.first.affected.first().ranges!!.first().events.any { e -> e.fixed != null }
}

DISPLAY(singularKbGhsaData.count())
//DISPLAY(singularKbGhsaData.toList().random())

255

In [20]:
singularKbGhsaData.map {
    object {
        val cveId = it.key
        val vulnIntroduced = it.value.first.affected.first().ranges!!.first().events.first { e -> e.introduced != null }.introduced
        val vulnFixed = it.value.first.affected.first().ranges!!.first().events.first { e -> e.fixed != null }.fixed
        val ghsaId = it.value.first.id
        val sapKbId = it.value.second.vulnerabilityID
    }
}.toDataFrame()

<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-1493172224"></table>

<p class="dataframe_description">... showing only top 20 of 255 rows</p><p class="dataframe_description">DataFrame: rowsCount = 255, columnsCount = 5</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for 

In [37]:
import com.fasterxml.jackson.databind.DeserializationFeature
import com.fasterxml.jackson.dataformat.xml.XmlMapper
import com.fasterxml.jackson.dataformat.xml.annotation.JacksonXmlProperty
import io.ktor.client.*
import io.ktor.client.call.*
import io.ktor.client.engine.cio.*
import io.ktor.client.plugins.*
import io.ktor.client.request.*
import kotlinx.coroutines.*

class MavenMetadata {
    lateinit var groupId: String
    lateinit var artifactId: String
    lateinit var versioning: MavenVersioning
}

class MavenVersioning {
    lateinit var latest: String
    lateinit var release: String
    lateinit var versions: List<String>
    lateinit var lastUpdated: String
}

fun packageNameToUrl(packageName: String): String {
    val parts = packageName.split(":")
    return parts[0].replace('.', '/') + "/" + parts[1]
}

val xmlMapper = XmlMapper()
xmlMapper.configure(DeserializationFeature.FAIL_ON_UNKNOWN_PROPERTIES, false)

val httpClient = HttpClient() {
    install(HttpTimeout) {
        requestTimeoutMillis = 120_000
    }
}

fun getVersions(packageName: String): List<String>? = runBlocking {
    val mavenRepoUrl = "https://repo1.maven.org/maven2"
    val jenkinsRepoUrl = "https://repo.jenkins-ci.org/releases"
    val result = httpClient
        .get("$mavenRepoUrl/${packageNameToUrl(packageName)}/maven-metadata.xml")
        .let {
            if (it.status.value == 200) {
                return@let it
            }
            println("http error ${it.status.value} for maven:$packageName")
            val result = httpClient.get("$jenkinsRepoUrl/${packageNameToUrl(packageName)}/maven-metadata.xml")
            result
        }.let {
            if (it.status.value == 200) {
                return@let it
            }
            println("http error ${it.status.value} for jenkins:$packageName")
            return@runBlocking null
        }

    val resultXml: String = result.body()
    try {
        val mvnMetaData = xmlMapper.readValue<MavenMetadata>(resultXml, MavenMetadata::class.java)
        val versionList = mvnMetaData.versioning.versions
        return@runBlocking versionList
    } catch (e: Exception) {
        println("parsing error for $packageName")
        throw e
    }
}

val dispatcher = Dispatchers.IO.limitedParallelism(8)

var packageToVersionList = runBlocking {
    singularKbGhsaData.map {
        async(dispatcher) {
            val packageName = it.value.first.affected.first().affectedPackage.name
            val versionList = getVersions(packageName)
            Pair(packageName, versionList)
        }
    }.awaitAll()
}.toMap()

DISPLAY(packageToVersionList.count())
DISPLAY(packageToVersionList.filter { it.value == null }.count())
DISPLAY(packageToVersionList.filter { it.value != null }.map { it.value!!.count() }.sum())
packageToVersionList = packageToVersionList.filter { it.value != null }

http error 404 for maven:com.github.blynkkk:blynk-server
http error 404 for jenkins:com.github.blynkkk:blynk-server
http error 404 for maven:com.walmartlabs.concord.docker:concord-common
http error 404 for jenkins:com.walmartlabs.concord.docker:concord-common
http error 404 for maven:org.xwiki.platform:xwiki-platform-web
http error 404 for jenkins:org.xwiki.platform:xwiki-platform-web
http error 404 for maven:org.apache.kafka:kafka
http error 404 for maven:org.jenkins-ci.plugins:mailer
http error 404 for maven:io.jenkins.blueocean:blueocean
http error 404 for jenkins:org.apache.kafka:kafka
http error 404 for maven:org.jenkins-ci.main:jenkins-core
http error 404 for maven:org.jenkins-ci.plugins:dynatrace-dashboard
http error 404 for maven:org.jenkins-ci.plugins:script-security
http error 404 for maven:io.jenkins.blueocean:blueocean
http error 404 for maven:org.jenkins-ci.plugins:git
http error 404 for maven:org.jenkins-ci.plugins:workflow-remote-loader
http error 404 for maven:org.jenki

167

4

14485

last output:
http error 404 for maven:com.github.blynkkk:blynk-server
http error 404 for jenkins:com.github.blynkkk:blynk-server
http error 404 for maven:org.jenkins-ci.plugins:script-security
http error 404 for maven:org.jenkins-ci.plugins:dynatrace-dashboard
http error 404 for maven:org.jenkins-ci.plugins:workflow-remote-loader
http error 404 for maven:org.jenkins-ci.plugins:dynatrace-dashboard
http error 404 for maven:org.jenkins-ci.main:jenkins-core
http error 404 for maven:org.jenkins-ci.plugins:git
http error 404 for maven:org.jenkins-ci.plugins:script-security
http error 404 for maven:io.jenkins.blueocean:blueocean
http error 404 for maven:org.apache.kafka:kafka
http error 404 for jenkins:org.apache.kafka:kafka
http error 404 for maven:io.jenkins.blueocean:blueocean
http error 404 for maven:org.jenkins-ci.plugins:mailer
http error 404 for maven:org.xwiki.platform:xwiki-platform-web
http error 404 for maven:com.walmartlabs.concord.docker:concord-common
http error 404 for jenkins:org.xwiki.platform:xwiki-platform-web

167
4
14477

only 4 errors!
in total we have 14474 versions for 167 packages

In [38]:
// it will return the same version if it is the final version
// it will return null if the version is not found in the version list
// it assumes that the packageName exists in packageToVersionList
fun getNextVersion(packageName: String, version: String): String? {
    val versionList = packageToVersionList[packageName]!!
    val versionIndex = versionList.indexOf(version)
    if (versionIndex == -1) return null
    if (versionIndex == versionList.count() - 1) return version
    return versionList[versionIndex + 1] 
}

fun getPrevVersion(packageName: String, version: String): String? {
    val versionList = 
        try {
            packageToVersionList[packageName]!!
        } catch (e: Exception) {
            println("$packageName not in the version list!")            
            throw e
        }
    val versionIndex = versionList.indexOf(version)
    if (versionIndex == -1) return null
    if (versionIndex == 0) return version
    return versionList[versionIndex - 1]
}

In [39]:
singularKbGhsaData
    .filter {
        val pkgName = it.value.first.affected.first().affectedPackage.name
        packageToVersionList.keys.contains(pkgName)
    }
    .map {
        val fixedVer = it.value.first.affected.first().ranges!!.first().events.first { it.fixed != null }.fixed!!
        val latestVulnVer = getPrevVersion(it.value.first.affected.first().affectedPackage.name, fixedVer)
        if (latestVulnVer != null && latestVulnVer == fixedVer) {
            println("latestVulnVer == fixedVer for ${it.value.first.affected.first().affectedPackage.name}")
        }
        latestVulnVer
    }.filter {
        it == null
    }.count()

47

47 packages version are not found in the version list from maven
204 records left

In [40]:
import org.apache.commons.csv.CSVFormat

data class InterimResult1(
    val cveId: String,
    val ghsaId: String,
    val packageName: String,
    val vulnIntroduced: String,
    val latestVulnVersion: String,
    val vulnFixed: String,
    val fixCommitRepo: String,
    val fixCommitHash: String,
)

val interimResult =
    singularKbGhsaData
        .filter {
            val pkgName = it.value.first.affected.first().affectedPackage.name
            packageToVersionList.keys.contains(pkgName)
        }
        .map {
            val fixedVer = it.value.first.affected.first().ranges!!.first().events.first { it.fixed != null }.fixed!!
            val latestVulnVer = getPrevVersion(it.value.first.affected.first().affectedPackage.name, fixedVer)
            if (latestVulnVer == null || latestVulnVer == fixedVer)
                return@map null

            val fixCommit = it.value.second.fixes!!.first().commits.first()

            InterimResult1(
                cveId = it.key,
                vulnIntroduced = it.value.first.affected.first().ranges!!.first().events.first { e -> e.introduced != null }.introduced!!,
                vulnFixed = it.value.first.affected.first().ranges!!.first().events.first { e -> e.fixed != null }.fixed!!,
                ghsaId = it.value.first.id,
                latestVulnVersion = latestVulnVer,
                fixCommitRepo = fixCommit.repository.let { it.replace(Regex("\\.git$"), "") },
                fixCommitHash = fixCommit.id,
                packageName = it.value.first.affected.first().affectedPackage.name,
            )
        }.filterNotNull()
        .toList()

DISPLAY(interimResult.count())
//interimResult.toDataFrame().writeCSV(dataDir.resolve("interim").also { it.mkdirs() }.resolve("interim_result.1.tsv"), format = CSVFormat.DEFAULT.withDelimiter('\t'))

204

In [41]:
val interimResultMap = interimResult
    .groupBy {
        it.cveId
    }.map {
        it.key to it.value.first()
    }.toMap()
interimResultMap.count()

204

In [42]:
import kotlin.io.path.exists

val sourceChanges = k.kbDataProvider.getSourceChanges()
    .filter { interimResultMap.containsKey(it.cveId) }
    .filterNot { it.beforeDir.exists() && it.afterDir.exists() }
    .toList()

DISPLAY(sourceChanges.count())
DISPLAY(sourceChanges.random())

73

SourceChange(cveId=CVE-2013-4310, commitHash=0c8366cb792227d484b9ca13e537037dd0cb57dc, beforeDir=D:\project-kb\statements\CVE-2013-4310\0c8366cb792227d484b9ca13e537037dd0cb57dc\before, afterDir=D:\project-kb\statements\CVE-2013-4310\0c8366cb792227d484b9ca13e537037dd0cb57dc\after)

There are 73 records that do not have source changes.
I download the patches from remote git repositories in the next notebook.  